# This example demonstrate how to train region classification (standart image classification model).

In [2]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [3]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Nomeroff-Net path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [4]:
# auto download latest dataset
from NomeroffNet.tools import modelhub

# auto download latest dataset
#info = modelhub.download_dataset_for_model("numberplate_options")
#PATH_TO_DATASET = info["dataset_path"]

# local path dataset
PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/OptionsDetector/numberplate_options_example")

In [5]:
PATH_TO_DATASET

'/mnt/data/var/www/nomeroff-net/./data/dataset/OptionsDetector/numberplate_options_example'

In [6]:
DATASET_NAME = "options"
VERSION = "2021_07_08_pytorch_lightning"

RESULT_PATH = os.path.join(NOMEROFF_NET_DIR, "./data/models/", 'numberplate_{}_{}.ckpt'.format(DATASET_NAME, VERSION))

In [10]:
from NomeroffNet import OptionsDetector

# definde your parameters
class MyNpClassificator(OptionsDetector):
    def __init__(self):
        OptionsDetector.__init__(self)
        
        # outputs 2
        self.class_region = [
            "xx-unknown", 
            "eu-ua-2015", 
            "eu-ua-2004", 
            "eu-ua-1995", 
            "eu", 
            "xx-transit", 
            "ru", 
            "kz", 
            "eu-ua-ordlo-dnr", 
            "eu-ua-ordlo-lnr", 
            "ge", 
            "by", 
            "su", 
            "kg"
        ]
        
        # output 3
        self.count_lines = ["0", "1", "2", "3"]
        
        self.epochs           = 1
        self.batch_size       = 1
        
        # count gpu for train
        self.gpus = 0
        
        self.height         = 64
        self.width         = 295


In [11]:
# initialize region detector.
npClassificator = MyNpClassificator()
npClassificator.prepare(PATH_TO_DATASET, verbose=1)

START PREPARING
DATA PREPARED


In [12]:
# train
model = npClassificator.train()


[INFO] best model path /mnt/data/var/www/nomeroff-net/train/lightning_logs/version_36/checkpoints/epoch=0-step=0.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 1.0, 'test_loss': 1.2801380157470703}
--------------------------------------------------------------------------------


In [13]:
npClassificator.prepare(PATH_TO_DATASET, verbose=1)

START PREPARING
DATA PREPARED


In [14]:
npClassificator.test()


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 1.0, 'test_loss': 1.2801380157470703}
--------------------------------------------------------------------------------


[{'test_loss': 1.2801380157470703, 'test_accuracy': 1.0}]

In [15]:
npClassificator.save(RESULT_PATH)

model save to /mnt/data/var/www/nomeroff-net/./data/models/numberplate_options_2021_07_08_pytorch_lightning.ckpt


# Classification Report

In [16]:
npClassificator.load(RESULT_PATH)

NPOptionsNet(
  (inp_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_reg): Dropout(p=0.2, inplace=False)
  (fc1_reg): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_reg): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_reg): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_reg): Linear(in_features=256, out_features=14, bias=True)
  (dropout_line): Dropout(p=0.2, inplace=False)
  (fc1_line): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_line): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_line): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_line): Linear(

In [17]:
from NomeroffNet.data_modules.option_img_generator import ImgGenerator
from sklearn.metrics import classification_report
import torch

def options_classification_report(npClassificator, 
                                  path_dir=os.path.join(PATH_TO_DATASET, "test"), 
                                  mode_torch="gpu"):
    imageGenerator = ImgGenerator(
        path_dir,
        npClassificator.width,
        npClassificator.height,
        npClassificator.batch_size,
        [len(npClassificator.class_region), len(npClassificator.count_lines)])
    imageGenerator.build_data()
    gen = imageGenerator.path_generator()
    
    out_idx_reg_all = []
    out_idx_line_all = []
    label_idx_reg_all = []
    label_idx_line_all = []
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
            npClassificator.model.cuda()
        with torch.no_grad():
            outputs = npClassificator.model(inputs)
        label_reg = torch.from_numpy(labels[0])
        label_cnt = torch.from_numpy(labels[1])
        if mode_torch == "gpu":
            label_reg = label_reg.cuda()
            label_cnt = label_cnt.cuda()
        
        out_idx_reg = torch.max(outputs[0], 1)[1].cpu().numpy()
        out_idx_line = torch.max(outputs[1], 1)[1].cpu().numpy()
        
        label_idx_reg = torch.max(label_reg, 1)[1].cpu().numpy()
        label_idx_line = torch.max(label_cnt, 1)[1].cpu().numpy()
        
        out_idx_reg_all.extend(out_idx_reg)
        out_idx_line_all.extend(out_idx_line)
        label_idx_reg_all.extend(label_idx_reg)
        label_idx_line_all.extend(label_idx_line)
    #print("out_idx_reg_all", out_idx_reg_all)
    print("\n\nclass region classification report:")
    print(classification_report(label_idx_reg_all, 
                                out_idx_reg_all, 
                                target_names=npClassificator.class_region))
    print("\n\ncount lines classification report:")
    print(classification_report(label_idx_line_all, 
                                out_idx_line_all, 
                                target_names=npClassificator.count_lines))

ModuleNotFoundError: No module named 'NomeroffNet.data_modules.option_img_generator'

In [18]:
options_classification_report(npClassificator)

NameError: name 'options_classification_report' is not defined

# Check small accuracy images

In [12]:
npClassificator.load(RESULT_PATH)

NPOptionsNet(
  (inp_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_reg): Dropout(p=0.2, inplace=False)
  (fc1_reg): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_reg): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_reg): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_reg): Linear(in_features=256, out_features=15, bias=True)
  (dropout_line): Dropout(p=0.2, inplace=False)
  (fc1_line): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_line): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_line): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_line): Linear(

In [13]:
%matplotlib inline

In [20]:
from NomeroffNet.data_modules.option_img_generator import ImgGenerator
import torch
import json
import cv2
import matplotlib.pyplot as plt

def show_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

def small_acc(npClassificator, 
              path_dir=os.path.join(PATH_TO_DATASET, "test"), 
              mode_torch="gpu", 
              min_acc=0.75):
    imageGenerator = ImgGenerator(
        path_dir,
        npClassificator.width,
        npClassificator.height,
        npClassificator.batch_size,
        [len(npClassificator.class_region), len(npClassificator.count_lines)])
    print("start imageGenerator build")
    imageGenerator.build_data()
    print("end imageGenerator build")
    gen = imageGenerator.path_generator()
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
        outputs = npClassificator.model(inputs)
        label_reg = torch.from_numpy(labels[0])
        label_cnt = torch.from_numpy(labels[1])
        if mode_torch == "gpu":
            label_reg = label_reg.cuda()
            label_cnt = label_cnt.cuda()
        out_idx_reg = torch.max(outputs[0], 1)[1].cpu().numpy()
        out_idx_line = torch.max(outputs[1], 1)[1].cpu().numpy()
        
        label_idx_reg = torch.max(label_reg, 1)[1].cpu().numpy()
        label_idx_line = torch.max(label_cnt, 1)[1].cpu().numpy()
        
        # check reg
        for o, l, p, tens in zip(out_idx_reg, label_idx_reg, img_paths, outputs[0]):
            if o != l:
                print("[WRONG REGION PREDICTED] pred", o, l, p)
                show_img(p)
            elif tens[l] < min_acc:
                print("[SMALL ACC REGION PREDICTED]  pred", o, l, tens[l], p)
                show_img(p)
        # check line
#         for o, l, p, tens in zip(out_idx_line, label_idx_line, img_paths, outputs[1]):
#             if o != l:
#                 print("[WRONG LINE PREDICTED]  pred", o, l, p)
#                 show_img(p)
#             elif tens[l] < min_acc:
#                 print("[SMALL ACC LINE PREDICTED]  pred", o, l, tens[l], p)
#                 show_img(p)

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "test"))

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "train"))

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "val"))